In [1]:


try:
  import warnings
  warnings.filterwarnings('ignore')
  import pandas as pd
  from pandas import json_normalize
  import numpy as np
  from plotnine import *
  import statsmodels.api as sm
  import sys
  from sklearn.linear_model import LogisticRegression 
  from sklearn.preprocessing import StandardScaler 
  from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
  from sklearn.model_selection import train_test_split
  from sklearn.model_selection import KFold
  from sklearn.model_selection import LeaveOneOut
  import seaborn as sns
  import csv
  %matplotlib inline
  from riotwatcher import LolWatcher, ApiError
except ImportError:
  print('Modules not installed')
  install('riotwatcher')
  install('pandas')
  install('sklearn')
  install('matplotlib')
  install('plotnine')
  install('statsmodels')
  install('seaborn')


# versions = lol_watcher.data_dragon.versions_for_region(my_region)
# champions_version = versions['n']['champion']
# summoner_spells_version = versions['n']['summoner']
# items_version = versions['n']['item']
# current_champ_list = lol_watcher.data_dragon.champions(champions_version)

def stringToList(string):
  listRes = list(string.split(", "))
  return(listRes)

lol_watcher = LolWatcher('RGAPI-b063e3e8-d1ce-433b-8f49-649ebd4ea78e')

user_region = 'na1'
modified_region = "americas"

me = input('Please enter your name: ')

def summoner(username):

  user_puuid = []
  user_id = []
  user_account_id = []
  user_mastery = []

  try:
    userInfo = lol_watcher.summoner.by_name(user_region, username)
    user_ranked_stats = lol_watcher.league.by_summoner(user_region, userInfo['id'])
    print('Analyzing data for', username)
  except:
    print(username, 'is not a valid summoner name.')
    sys.exit(1)
  
  df_summoner = pd.DataFrame(user_ranked_stats, columns = ['leagueId', 'queueType', 'tier', 'rank', 'summonerId', 
    'summonerName', 'leaguePoints', 'wins', 'losses', 'veteran', 'inactive', 'freshBlood', 'hotStreak'])
  
  df_summoner_cleaned = df_summoner[df_summoner['queueType'] == 'RANKED_SOLO_5x5']

  try:
    summoner_name = df_summoner_cleaned.iat[0,5]
    summoner_id = df_summoner_cleaned.iat[0,4]
    tier = df_summoner_cleaned.iat[0,2]
    rank = df_summoner_cleaned.iat[0,3]
    lp = df_summoner_cleaned.iat[0,6]
    ranked_wins = df_summoner_cleaned.iat[0,7]
    ranked_losses = df_summoner_cleaned.iat[0,8]
    total_games = int(ranked_wins) + int(ranked_losses)
    win_loss_ratio = round((int(ranked_wins) / total_games) * 100, 2)
    target = df_summoner_cleaned.iat[0,4]
  except:
    print('Error in indexing summoner data for', username)
    print(username, 'may not have enough ranked games played or finished provisionals.')
    sys.exit(2)
  
  try:
    print(summoner_name, 'is currently', tier, rank, lp, 'lp', 'with a win rate of', win_loss_ratio, '%', 'in', total_games, 'ranked games.')
  except:
    pass

  try:
    summoner_encrypted = lol_watcher.summoner.by_id(user_region, target)
    puuid = summoner_encrypted['puuid']
    summonerId = summoner_encrypted['id']
    summoner_account_Id = summoner_encrypted['accountId']
    user_puuid.append(puuid)
    user_id.append(summonerId)
    user_account_id.append(summoner_account_Id)
    user_mastery = lol_watcher.champion_mastery.by_summoner(user_region, user_id[0])
    champ_mastery_df = pd.DataFrame(user_mastery)
    champ_mastery_df.to_csv('user_masteries.csv')
  except:
    print('Unable to get champion masteries for', username)
    sys.exit(3)
  
  try:
    my_match_ids = []
    me = lol_watcher.summoner.by_name(user_region, username)
    my_matches = lol_watcher.match.matchlist_by_puuid(modified_region, user_puuid[0])
    for gameId in my_matches:
      my_match_ids.append(gameId)
  except:
    print('Error in grabbing match IDs for', username)
    sys.exit(4)

  # try:
  #   m1 = my_match_ids[0]
  #   m2 = my_match_ids[1]
  #   m3 = my_match_ids[2]
  #   m4 = my_match_ids[3]
  #   m5 = my_match_ids[4]
  #   m6 = my_match_ids[5]
  #   m7 = my_match_ids[6]
  #   m8 = my_match_ids[7]
  #   m9 = my_match_ids[8]
  #   m10 = my_match_ids[9]
  #   m11 = my_match_ids[10]
  #   m12 = my_match_ids[11]
  #   m13 = my_match_ids[12]
  #   m14 = my_match_ids[13]
  #   m15 = my_match_ids[14]
  #   m16 = my_match_ids[15]
  #   m17 = my_match_ids[16]
  #   m18 = my_match_ids[17]
  #   m19 = my_match_ids[18]
  #   m20 = my_match_ids[19]
  #   m1_history = lol_watcher.match.by_id(modified_region, m1)
  #   m2_history = lol_watcher.match.by_id(modified_region, m2)
  #   m3_history = lol_watcher.match.by_id(modified_region, m3)
  #   m4_history = lol_watcher.match.by_id(modified_region, m4)
  #   m5_history = lol_watcher.match.by_id(modified_region, m5)
  #   m6_history = lol_watcher.match.by_id(modified_region, m6)
  #   m7_history = lol_watcher.match.by_id(modified_region, m7)
  #   m8_history = lol_watcher.match.by_id(modified_region, m8)
  #   m9_history = lol_watcher.match.by_id(modified_region, m9)
  #   m10_history = lol_watcher.match.by_id(modified_region, m10)
  #   m11_history = lol_watcher.match.by_id(modified_region, m11)
  #   m12_history = lol_watcher.match.by_id(modified_region, m12)
  #   m13_history = lol_watcher.match.by_id(modified_region, m13)
  #   m14_history = lol_watcher.match.by_id(modified_region, m14)
  #   m15_history = lol_watcher.match.by_id(modified_region, m15)
  #   m16_history = lol_watcher.match.by_id(modified_region, m16)
  #   m17_history = lol_watcher.match.by_id(modified_region, m17)
  #   m18_history = lol_watcher.match.by_id(modified_region, m18)
  #   m19_history = lol_watcher.match.by_id(modified_region, m19)
  #   m20_history = lol_watcher.match.by_id(modified_region, m20)
    
  #   m1_game = []
  #   m2_game = []
  #   m3_game = []
  #   m4_game = []
  #   m5_game = []
  #   m6_game = []
  #   m7_game = []
  #   m8_game = []
  #   m9_game = []
  #   m10_game = []
  #   m11_game = []
  #   m12_game = []
  #   m13_game = []
  #   m14_game = []
  #   m15_game = []
  #   m16_game = []
  #   m17_game = []
  #   m18_game = []
  #   m19_game = []
  #   m20_game = []

  #   m1_game.append(m1_history)
  #   m2_game.append(m2_history)
  #   m3_game.append(m3_history)
  #   m4_game.append(m4_history)
  #   m5_game.append(m5_history)
  #   m6_game.append(m6_history)
  #   m7_game.append(m7_history)
  #   m8_game.append(m8_history)
  #   m9_game.append(m9_history)
  #   m10_game.append(m10_history)
  #   m11_game.append(m11_history)
  #   m12_game.append(m12_history)
  #   m13_game.append(m13_history)
  #   m14_game.append(m14_history)
  #   m15_game.append(m15_history)
  #   m16_game.append(m16_history)
  #   m17_game.append(m17_history)
  #   m18_game.append(m18_history)
  #   m19_game.append(m19_history)
  #   m20_game.append(m20_history)
  # except:
  #   print('There was an error in grabbing the past 20 matches for ', username)
  #   sys.exit(3)

  games_to_analyze = []
  participants = []
  games_to_filter = [m1_game, m2_game, m3_game, m4_game, m5_game, m6_game, m7_game, m8_game, m9_game, m10_game,
   m11_game, m12_game, m13_game, m14_game, m15_game, m16_game, m17_game, m18_game, m19_game, m20_game]
  
  try:
    for x in games_to_filter:
      df = pd.DataFrame(x)
      iteration = df['info']
      indexing_iteration = iteration[0]
      final_index = indexing_iteration['participants']
      if indexing_iteration['queueId'] == 420:
        games_to_analyze.append(x)
        for row in final_index:
          participants_row = {}
          print(row['summonerName'])
          if row['summonerName'] == username:
            participants_row['summoner'] = row['summonerName']
            participants_row['champion'] = row['championName']
            participants_row['kills'] = row['kills']
            participants_row['deaths'] = row['deaths']
            participants_row['assists'] = row['assists']
            participants_row['totalDamagetoChampions'] = row['totalDamageDealtToChampions']
            participants_row['position'] = row['teamPosition']
            participants_row['win'] = row['win']
            participants.append(participants_row)
        else:
          pass
    else:
      pass
  except:
    print('Error in parsing match data of', username, 'to DataFrame.')
    sys.exit(5)

summoner(me)

Analyzing data for test
Error in indexing summoner data for test
test may not have enough ranked games played or finished provisionals.


AssertionError: 